## Examples of `degirum_tools.streams` module usage 

This notebook contains various examples, which demonstrate the capabilities of `degirum_tools.streams` module: streaming toolkit for PySDK.

The script also uses either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified in [env.ini](../../env.ini) file by defining `CAMERA_ID` variable. If not specified, default local camera will be used.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools
import degirum_tools
degirum_tools.configure_colab() # configure for Google Colab

### Simplest example: video source is connected to video display

In [ ]:
from degirum_tools.streams import *

# create Composition: an object, which keeps all streaming blocks (called gizmos) and runs them in parallel
c = Composition()

# create gizmos:
source = VideoSourceGizmo()  # video source gizmo
display = VideoDisplayGizmo(
    "press `x` or `q` to stop", allow_drop=False
)  # video display gizmo

# Create pipeline: connect display input to source output
display.connect_to(source)

# add gizmos to composition
c.add(source)
c.add(display)

# start composition
c.start()

# call c.stop() to stop composition or just press `x` or `q` in display window


### Same example, but with compact syntax

In [ ]:
from degirum_tools.streams import *

c = Composition()

# Create gizmos and pipeline as a single-liner:
# we use __call__() operator of Composition class instead of add() method
# and we use `>>` operator of gizmo classes instead of connect_to() method
c(VideoSourceGizmo()) >> c(VideoDisplayGizmo())

c.start()


### Example of forked streams

In [ ]:
from degirum_tools.streams import *

c = Composition()

# create and add to composition all required gizmos
source = c.add(VideoSourceGizmo())  # video source gizmo
display = c.add(
    VideoDisplayGizmo(["Original", "Resized"])
)  # two-input display gizmo: will show two windows
resizer = c.add(ResizingGizmo(300, 200))  # resizing gizmo

# Create pipeline: the image source is fed to a display and to the image resizing gizmo,
# which is then fed to another display.

display.connect_to(source, 0)  # display input 0 is "Original"
resizer.connect_to(source)
display.connect_to(resizer, 1)  # display input 1 is "Resized"

c.start()


### Example of capturing video input into file with simultaneous display on a screen

In [ ]:
from degirum_tools.streams import *

c = Composition()

source = c.add(VideoSourceGizmo())
display = c.add(VideoDisplayGizmo())
saver = c.add(VideoSaverGizmo("../../workarea/mycapture.mp4"))

source >> display
source >> saver

c.start()


### Example of AI inference from the camera with AI inference results display

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

In [ ]:
import degirum as dg, degirum_tools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = degirum_tools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, degirum_tools.get_cloud_zoo_url(), degirum_tools.get_token())


from degirum_tools.streams import *

# load some object detection AI model
model = zoo.load_model("yolo_v5s_coco--512x512_quant_n2x_orca1_1")

c = Composition()

# create gizmos
source = c.add(VideoSourceGizmo())  # video source
detection = c.add(AiSimpleGizmo(model))  # AI model
display = c.add(
    VideoDisplayGizmo("Detection", show_ai_overlay=True, show_fps=True)
)  # display

# create pipeline
source >> detection >> display

c.start()


### Example of AI inference with separate pre-processing stage

Frames from the camera are supplied to the object detection model preprocessor and then to the object detection model
itself. This improves performance to let image resizing preprocessor running is separate thread.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

In [ ]:
import degirum as dg, degirum_tools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = degirum_tools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, degirum_tools.get_cloud_zoo_url(), degirum_tools.get_token())


from degirum_tools.streams import *

# load AI model
model = zoo.load_model("yolo_v5s_coco--512x512_quant_n2x_orca1_1")

c = Composition()

# create gizmos
source = c.add(VideoSourceGizmo())  # video source
preprocessor = c.add(AiPreprocessGizmo(model))
detection = c.add(AiSimpleGizmo(model))
display = c.add(VideoDisplayGizmo("Objects", show_ai_overlay=True, show_fps=True))  # display

# create pipeline
source >> preprocessor >> detection >> display

c.start()